In [127]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
from sklearn.metrics import confusion_matrix, f1_score, precision_score,\
recall_score, confusion_matrix, classification_report, accuracy_score 
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
def __GetLIWC(file:str): 
	liwc = pd.read_csv(file)
	liwc = liwc.rename(columns = {liwc.columns[2]:'user_id'})
	liwcUser = liwc.groupby('user_id').mean().reset_index()
	liwcUser = liwcUser.drop(['Source (A)', 'Source (D)'], axis=1)
	return liwcUser

def mergeFea(features, liwc, empath): 
	features = pd.read_csv(features)

	#merge features
	liwcUser = __GetLIWC(liwc)
	liwcUser2 = liwcUser.iloc[:,1::]
	liwcUser2.columns = [str(col) + '_liwc' for col in liwcUser2.columns]
	liwcUser2['user_id'] = liwcUser.user_id

	empath = pd.read_csv(empath)
	empath2 = empath.iloc[:,1::]
	empath2.columns = [str(col) + '_empath' for col in empath2.columns]
	empath2['user_id'] = empath.user_id

	allfea = pd.merge(features, liwcUser2, on = 'user_id', how = 'right')
	allfea = pd.merge(allfea, empath2, on = 'user_id', how = 'right')
	return allfea

def preprocess2(sent):
    #remove punctustion
    sent = re.sub(r'[^\w\s]','',str(sent))
    words = sent.split()
    new_words = []
    for w in words:      
        new_words.append(w.lower())
        
    return ' '.join(new_words)


class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key].values.reshape(-1,1)
    
class ItemSelectorText(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [317]:
obj = ItemSelector('motivations')
obj.transform(X_train)

array([[4],
       [0],
       [4],
       [1],
       [0],
       [3],
       [3],
       [5],
       [5],
       [0],
       [3],
       [5],
       [2],
       [5],
       [1],
       [1],
       [4],
       [5],
       [2],
       [0],
       [5],
       [5],
       [5],
       [5],
       [4],
       [2],
       [5],
       [5],
       [3],
       [5],
       [0],
       [2],
       [3],
       [5],
       [5],
       [5],
       [3],
       [5],
       [1],
       [4],
       [5],
       [2],
       [4],
       [4],
       [4],
       [0],
       [5],
       [1],
       [4],
       [4],
       [5],
       [3],
       [5],
       [2],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [2],
       [2],
       [3],
       [5],
       [4],
       [5],
       [5],
       [5],
       [4],
       [4],
       [2],
       [5],
       [5],
       [4],
       [2],
       [4],
       [5],
       [3],
       [2],
       [3],
       [4],
    

In [5]:
#path = '/home/lucia/phd_work/shareTask/'
path = '/Users/lucia/phd_work/Clpsy/'
X = pd.read_csv(path + '/data/clpsych19_training_data/Btrain_NoNoise_SW.csv')
y = pd.read_csv(path + '/data/clpsych19_training_data/crowd_train.csv')

In [8]:
#concatenate text according to user id
X1 = X[['user_id','post_body']]
conTex = X1.groupby(['user_id'],as_index=False).agg(lambda x : x.sum() if str(x) else ' '.join(x))


In [9]:
conTex['post_body'] = conTex['post_body'].apply(lambda x: preprocess2(x))

In [106]:
Fea = pd.read_csv(path + '/suicideDetection/features/FreqSentiMotiTopiFea.csv')
Fea.columns

Index(['Unnamed: 0', 'user_id', 'raw_label', 'postingFrequency',
       'postingInterval', 'generalMoreFreq', 'generalWordCount',
       'healthPostingFrequency', 'healthPostingInterval', 'healthMoreFreq',
       'healthWordCount', 'mentionMethods', 'SWFrequency', 'SWPostingInterval',
       'SWFreq', 'SWWordCount', 'fin_body', 'drug_body', 'mental_body',
       'rela_body', 'suicide_body', 'hopeless_body', 'motivations',
       'family_senti', 'partner_senti', 'self_senti', 'raw_label.1',
       'sentiment', 'mclust'],
      dtype='object')

In [159]:
allFea = pd.read_csv(path + '/suicideDetection/features/FreqSentiMotiTopiFea.csv')
liwc = pd.read_csv(path + '/suicideDetection/features/liwcSW.csv')
tags = pd.read_csv(path + '/suicideDetection/features/TagFeaSW.csv')
empath = pd.read_csv(path + '/suicideDetection/features/empathSW.csv')
readability = pd.read_csv(path + '/suicideDetection/features/readability.csv')

liwc = liwc.iloc[:,np.r_[2, 8:liwc.shape[1]]]
liwc = liwc.rename(columns = {liwc.columns[0]:'user_id'})
liwcUser = liwc.groupby('user_id').mean().reset_index()
liwcUser.columns = [str(col) + '_liwc' for col in liwcUser.columns]
readability = readability.drop(['post_body'], axis = 1)

empath.columns = [str(col) + '_empath' for col in empath.columns]
tags.columns = [str(col) + '_tag' for col in tags.columns]
readability.columns = [str(col) + '_read' for col in readability.columns]
allFea = pd.merge(allFea, liwcUser, left_on ='user_id', right_on = 'user_id_liwc', how = 'left')
allFea = pd.merge(allFea, empath, left_on ='user_id', right_on = 'user_id_empath', how = 'left')
allFea = pd.merge(allFea, tags, left_on ='user_id', right_on = 'user_id_tag', how = 'left')
allFea = pd.merge(allFea, readability, left_on ='user_id', right_on = 'user_id_read', how = 'left')

# # # #liwc
# # # allFea


In [160]:
#fea = pd.merge(y, conTex, on = 'user_id')
allFea2  = pd.merge(conTex, allFea, on = 'user_id')
#X = fea['post_body']
y = allFea2.raw_label

In [161]:
X_train, X_test, y_train, y_test = train_test_split(allFea2, y, test_size=0.30, random_state=35)

In [162]:
print(X_train.shape)
print(y_train.shape)

(347, 183)
(347,)


In [163]:
X_train.columns

Index(['user_id', 'post_body', 'Unnamed: 0', 'raw_label', 'postingFrequency',
       'postingInterval', 'generalMoreFreq', 'generalWordCount',
       'healthPostingFrequency', 'healthPostingInterval',
       ...
       'VBN_tag', 'VBZ_tag', 'Unnamed: 0_read', 'user_id_read',
       'readEase_read', 'Flesch-Kincaid_read', 'gunning_fog_read',
       'smog_index_read', 'coleman_liau_read', 'linsear_write_read'],
      dtype='object', length=183)

In [185]:
get_liwc_data = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('liwc')]], validate=False)
get_empath_data = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('empath')]], validate=False)
get_tags = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('tag')]], validate=False)
get_read = FunctionTransformer(lambda x: x[x.columns[x.columns.to_series().str.contains('read')]], validate=False)


pipe1 = Pipeline([
    
    ('feats', FeatureUnion([
        ('text', Pipeline([
            ('selector', ItemSelectorText(key='post_body')),
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
             ])),
        ('liwc_features', Pipeline([
                ('selector', get_liwc_data) ])),
        ('empath_features', Pipeline([
                ('selector', get_empath_data)])),
        ('tag_features', Pipeline([
                ('selector', get_tags)])),
#         ('readability_features', Pipeline([
#                 ('selector', get_read)])),
        ('selector1', ItemSelector(key='motivations')),
#         ('selector2', ItemSelector(key='healthPostingFrequency')),
#         ('selector3', ItemSelector(key='healthPostingInterval')),
#         ('selector4', ItemSelector(key='healthMoreFreq')),
        ('selector5', ItemSelector(key='healthWordCount')),
        ('selector6', ItemSelector(key='mentionMethods')),
        ('selector7', ItemSelector(key='SWFrequency')),
#         ('selector8', ItemSelector(key='SWPostingInterval')),
#         ('selector9', ItemSelector(key='SWFreq')),
#         ('selector10', ItemSelector(key='suicide_body')),
#         ('selector11', ItemSelector(key='hopeless_body')),
#         ('selector12', ItemSelector(key='self_senti')),
#         ('selector13', ItemSelector(key='mclust')),
#         ('selector14', ItemSelector(key='family_senti')),
             ])),
    
       ('clf', Pipeline([
       ('scale', StandardScaler(with_mean=False)),
       ('feature_selection', SelectFromModel(RandomForestClassifier(random_state=20))),
#       ('svm',  svm.SVC())
       ('log', LogisticRegression()),
#      ('gbn', GaussianNB()),
         ])),
])

# parameters = [{'feature_selection__estimator__max_depth': [5,10,20], 'feature_selection__estimator__max_leaf_nodes': [50, 100, 200],
#             'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
#             'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0, 10] , 'svc__class_weight':['balanced']}]
parameters = [{'feature_selection__estimator__max_depth': [5,10,20], 'feature_selection__estimator__max_leaf_nodes': [50, 100, 200],
               'log__C':[1.0, 2.0, 3.0], 'clf__log__class_weight': ['balanced'], 'multi_class': ['ovr', 'multinomial']}]
    
    
estimator = GridSearchCV(pipe1, parameters, scoring='roc_auc')
pipe1.fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           a       0.75      0.69      0.72        35
           b       0.14      0.06      0.08        18
           c       0.29      0.25      0.27        32
           d       0.59      0.75      0.66        64

   micro avg       0.54      0.54      0.54       149
   macro avg       0.44      0.44      0.43       149
weighted avg       0.51      0.54      0.52       149



/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
÷

In [67]:
pipe1.fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           a       0.70      0.66      0.68        35
           b       0.33      0.11      0.17        18
           c       0.19      0.22      0.20        32
           d       0.55      0.62      0.58        64

   micro avg       0.48      0.48      0.48       149
   macro avg       0.44      0.40      0.41       149
weighted avg       0.48      0.48      0.47       149



/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [491]:
pipe1.steps

[('feats', FeatureUnion(n_jobs=None,
         transformer_list=[('text', Pipeline(memory=None,
       steps=[('selector', ItemSelectorText(key='post_body')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_f...body')), ('selector12', ItemSelector(key='self_senti')), ('selector13', ItemSelector(key='mclust'))],
         transformer_weights=None)), ('clf', Pipeline(memory=None,
       steps=[('scale', StandardScaler(copy=True, with_mean=False, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False))]))]

In [61]:
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           a       0.69      0.69      0.69        35
           b       0.20      0.06      0.09        18
           c       0.23      0.19      0.21        32
           d       0.55      0.72      0.63        64

   micro avg       0.52      0.52      0.52       149
   macro avg       0.42      0.41      0.40       149
weighted avg       0.47      0.52      0.48       149



check parameter names

In [307]:
pipe1.named_steps['logit'].get_params()

KeyError: 'logit'

In [52]:
pipe1.get_params()

{'clf': Pipeline(memory=None,
      steps=[('scale', StandardScaler(copy=True, with_mean=False, with_std=True)), ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decr...penalty='l2', random_state=None, solver='warn',
           tol=0.0001, verbose=0, warm_start=False))]),
 'clf__feature_selection': SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=20, verbose=0, warm_start=False),
         max_features=None, norm_order=1, prefit=False, threshold=None),
 'clf__fe

In [309]:
pipe1.set_params(cv__min_df=6, 
                 cv__lowercase=False).fit(X_train, y_train)
pred = pipe1.predict(X_test)
print(classification_report(y_test, pred))

ValueError: Invalid parameter cv for estimator Pipeline(memory=None,
     steps=[('feats', FeatureUnion(n_jobs=None,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('selector', ItemSelectorText(key='post_body')), ('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='c...alty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [175]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter

class PosTagMatrix(BaseEstimator, TransformerMixin):
    #normalise = True - devide all values by a total number of tags in the sentence
    #tokenizer - take a custom tokenizer function
    def __init__(self, tokenizer=lambda x: x.split(), normalize=True):
        self.tokenizer=tokenizer
        self.normalize=normalize

    #helper function to tokenize and count parts of speech
    def pos_func(self, sentence):
        return Counter(tag for word,tag in nltk.pos_tag(self.tokenizer(sentence)))

    # fit() doesn't do anything, this is a transformer class
    def fit(self, X, y = None):
        return self

    #all the work is done here
    def transform(self, X):
        X_tagged = X.apply(self.pos_func).apply(pd.Series).fillna(0)
        X_tagged['n_tokens'] = X_tagged.apply(sum, axis=1)
        if self.normalize:
            X_tagged = X_tagged.divide(X_tagged['n_tokens'], axis=0)

        return X_tagged

In [197]:
obj = PosTagMatrix()
tranTag = obj.transform(X_train)
tranTag.columns

Index(['CD', 'NNS', 'RB', 'NN', 'VBD', 'CC', 'VBG', 'IN', 'DT', 'PRP$', 'VBP',
       'JJ', 'PRP', 'VBN', 'VBZ', 'TO', 'VB', 'MD', 'RP', 'WP', 'WRB', 'JJR',
       'EX', 'RBR', 'JJS', 'WDT', 'PDT', 'RBS', 'NNP', 'FW', '$', 'UH', 'WP$',
       'n_tokens'],
      dtype='object')

In [ ]:
testTag = obj.transform(X_test)
tran.columns

In [ ]:
test_x2 = test_x.loc[:, test_x.columns.isin(tranTag)]

In [172]:
from sklearn.pipeline import FeatureUnion

pipe2 = Pipeline([
    ('u1', FeatureUnion([
        ('tfdif_features', Pipeline([
            ('cv', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])),
        ('pos_features', Pipeline([
            ('pos', PosTagMatrix(tokenizer=nltk.word_tokenize) ),
        ])),
    ])),
    ('logit', LogisticRegression()),

])

In [194]:
print(X_train.shape)
print(X_test.shape)

(347,)
(149,)


In [186]:
pipe2.fit(X_train, y_train)
pred = pipe2.predict(X_test)
#print(classification_report(y_test, pred))

/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


ValueError: X has 8634 features per sample; expecting 8635